In [1]:
!pip install -r requirements.txt
!pip install tensorflow
!pip install keras
!pip install pillow
!pip install --user --upgrade pip

Requirement already up-to-date: pip in c:\programdata\anaconda3\lib\site-packages (19.3.1)


In [2]:
# Convolutional Neural Network

# Part 1 - Building the CNN

# Importing the Keras libraries and packages
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from PIL import Image

Using TensorFlow backend.


In [3]:
# Initialising the CNN
classifier = Sequential()

# Step 1 - Convolution
classifier.add(Conv2D(32, (4, 4), input_shape = (64, 64, 3), activation = 'relu'))

# Step 2 - Pooling
classifier.add(MaxPooling2D(pool_size = (2, 2)))

# Adding a second convolutional layer
classifier.add(Conv2D(32, (4, 4), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))

# Adding a second convolutional layer
classifier.add(Conv2D(32, (4, 4), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))

# Adding a second convolutional layer
classifier.add(Conv2D(32, (4, 4), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))

# Step 3 - Flattening
classifier.add(Flatten())

# Step 4 - Full connection
classifier.add(Dense(units = 128, activation = 'relu'))
classifier.add(Dense(units = 1, activation = 'sigmoid'))

# Compiling the CNN
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])


Instructions for updating:
Colocations handled automatically by placer.


In [4]:
# Part 2 - Fitting the CNN to the images

from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory('dataset/training_set',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

test_set = test_datagen.flow_from_directory('dataset/test_set',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'binary')

classifier.fit_generator(training_set,
                         steps_per_epoch = 4000,
                         epochs = 20,
                         validation_data = test_set,
                         validation_steps = 2000)

Found 8000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.
Instructions for updating:
Use tf.cast instead.
Epoch 1/20
4000/4000 [==============================] - 703s 176ms/step - loss: 0.4939 - acc: 0.7482 - val_loss: 0.4285 - val_acc: 0.8040
Epoch 2/20
4000/4000 [==============================] - 621s 155ms/step - loss: 0.3325 - acc: 0.8518 - val_loss: 0.4278 - val_acc: 0.8061
Epoch 3/20
4000/4000 [==============================] - 648s 162ms/step - loss: 0.2642 - acc: 0.8853 - val_loss: 0.4452 - val_acc: 0.8243
Epoch 4/20
4000/4000 [==============================] - 648s 162ms/step - loss: 0.2199 - acc: 0.9070 - val_loss: 0.5037 - val_acc: 0.8238
Epoch 5/20
4000/4000 [==============================] - 639s 160ms/step - loss: 0.1909 - acc: 0.9207 - val_loss: 0.5086 - val_acc: 0.8315
Epoch 6/20
4000/4000 [==============================] - 678s 170ms/step - loss: 0.1702 - acc: 0.9299 - val_loss: 0.4928 - val_acc: 0.8297
Epoch 7/20
4000/4000 [==================

In [7]:
# Part 3 - Making new predictions

import numpy as np
from keras.preprocessing import image
test_image = image.load_img('dataset/single_prediction/cat_or_dog_1.jpg', target_size = (64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = classifier.predict(test_image)
training_set.class_indices
if result[0][0] == 1:
    prediction = 'dog'
else:
    prediction = 'cat'

[[1.]]
